## ETL -1

In [2]:
# librerias:
import requests
import pandas as pd
import numpy as np

# para ver más de una salida por celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

En el marco de un proyecto de análisis de datos, se requiere realizar una ETL para integrar información de distintas fuentes de datos. La empresa para la que trabajamos cuenta con una base de datos de la informacion sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde el año 1998 hasta el 2022. Quieren enriquecer los datos con información de la generación de energías renovables y no renovables y crear una base de datos más completa. Para esto nos piden que usemos la API de REData la cual nos da mucha información sobre el mercado energético en España. La documentación de la API la encontraréis aquí:

https://www.ree.es/es/apidatos

In [4]:
# definimos la url de la API
url = 'https://www.ree.es/es/apidatos'

In [5]:
# hacemos la petición de datos a la API
response = requests.get(url = url)

In [6]:
# comprobamos si la petición se está realizando correctamente
r_stat = response.status_code
r_reason = response.reason

print(f'El código devuelto por la API es {r_stat}, con razón: {r_reason}')

El código devuelto por la API es 200, con razón: OK


Requisitos de este ejercicio de pair programming:

1- Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable.

2- Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [ ]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

3- Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs